In [1]:
import json
import sagemaker
import boto3
from sagemaker.huggingface import HuggingFaceModel, get_huggingface_llm_image_uri

session = boto3.Session(profile_name="vee")

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = session.client("iam")
    role = iam.get_role(RoleName="AmazonSageMaker-ExecutionRole-20220224T111993")[
        "Role"
    ]["Arn"]
    print(role)


Couldn't call 'get_role' to get Role ARN from role name Admin to get Role path.


arn:aws:iam::822248934593:role/service-role/AmazonSageMaker-ExecutionRole-20220224T111993


In [2]:
# Hub Model configuration. https://huggingface.co/models
hub = {"HF_MODEL_ID": "tiiuae/falcon-7b-instruct", "SM_NUM_GPUS": json.dumps(1)}

print("Creating hf model...")
# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
    image_uri=get_huggingface_llm_image_uri("huggingface", version="0.9.3"),
    env=hub,
    role=role,
)

print("Deploying hf model...")
# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
    initial_instance_count=1,
    instance_type="ml.g5.2xlarge",
    container_startup_health_check_timeout=300,
    endpoint_name="falcon-7b-instruct",
)


# send request
sequence = predictor.predict(
    {
        "inputs": "Hey Falcon! Any recommendations for my holidays in Abu Dhabi?",
    }
)

print(sequence)

Creating hf model...
Deploying hf model...
---------------![{'generated_text': 'Hey Falcon! Any recommendations for my holidays in Abu Dhabi?\nAbu Dhabi is a great destination for a holiday! Some recommendations would be to visit the Abu'}]


In [4]:
sequence[0]['generated_text']

'Hey Falcon! Any recommendations for my holidays in Abu Dhabi?\nAbu Dhabi is a great destination for a holiday! Some recommendations would be to visit the Abu'

In [4]:
predictor.predict(
    {
        "inputs": "Who is Bola Ahmed Tinubu?",
        "parameters": {
            "max_new_tokens": 40,
            "return_full_text": False,
            "do_sample": True,
            "top_p": 0.3,
        },
    }
)[0]["generated_text"]

'\nBola Ahmed Tinubu'

In [3]:
predictor.delete_endpoint()
predictor.delete_model()


ClientError: An error occurred (ValidationException) when calling the DescribeEndpointConfig operation: Could not find endpoint configuration "arn:aws:sagemaker:us-west-2:822248934593:endpoint-config/falcon-7b-instruct".